# Dataset creation

In this Notebook, weather and traffic data related to the *[Adelaide City Council](https://www.google.com/maps/place/City+of+Adelaide,+South+Australia,+Australia/@-34.9203673,138.5801184,14z/data=!3m1!4b1!4m6!3m5!1s0x6ab0ced5a8ad9dcb:0xaf8b03f697530eb6!8m2!3d-34.9236716!4d138.6038129!16zL20vMGIyc3du?entry=ttu)* is merged and normalized.

In [1]:
import numpy as np
import pandas as pd

import holidays

import json
import folium
from collections import OrderedDict

from IPython.display import display

Import necessary CSV files

In [2]:
tf_data = pd.read_csv('../data/Traffic_Intersection/data.gz', compression='gzip', date_format=["date"]).reset_index(drop=True)
wt_data = pd.read_csv('../data/Weather/data.csv')

wt_data.drop(wt_data.columns[[0]], axis=1, inplace=True) # Drop del index "aggiuntivo"
wt_data['date'] = pd.to_datetime(wt_data.date).dt.tz_localize(None)
wt_data.head()

,date,temperature_2m,relative_humidity_2m,apparent_temperature,precipitation,wind_speed_10m,cloud_cover
0,2010-01-01 00:00:00,23.539999,52.424004,24.445896,0.0,7.862518,0.9
1,2010-01-01 01:00:00,25.490000,47.725876,26.860800,0.0,11.212135,0.0
2,2010-01-01 02:00:00,27.039999,44.841870,28.257778,0.0,16.489416,0.0
3,2010-01-01 03:00:00,27.789999,43.756590,28.697495,0.0,20.188908,0.0
4,2010-01-01 04:00:00,27.990000,43.670140,28.495106,0.0,22.406927,0.0


In [3]:
tf_data['date'] = pd.to_datetime(tf_data.date)
tf_data.head()

,site_no,site_desc,lat,lon,date,hourly_traffic,t_detector_counts,t_valid_counts,t_error_counts,error_ratio
0,1.0,SIGNALISED 4 WAY INTERSECTION,-34.921569,138.599436,2010-01-01 00:00:00,4167.0,288.0,288.0,0.0,0.0
1,1.0,SIGNALISED 4 WAY INTERSECTION,-34.921569,138.599436,2010-01-01 01:00:00,4000.0,288.0,288.0,0.0,0.0
2,1.0,SIGNALISED 4 WAY INTERSECTION,-34.921569,138.599436,2010-01-01 02:00:00,3172.0,288.0,288.0,0.0,0.0
3,1.0,SIGNALISED 4 WAY INTERSECTION,-34.921569,138.599436,2010-01-01 03:00:00,2558.0,288.0,288.0,0.0,0.0
4,1.0,SIGNALISED 4 WAY INTERSECTION,-34.921569,138.599436,2010-01-01 04:00:00,2386.0,288.0,288.0,0.0,0.0


### Creation of geoJSON related to 'siti_no' for map visualization

In [4]:
SiteLocation_data = tf_data.drop_duplicates(subset='site_no')
SiteLocation_data = SiteLocation_data[['site_no', 'site_desc', 'lat', 'lon']].set_index("site_no", drop=True)
SiteLocation_data.to_csv('../data/0_Site_table.csv')

SiteLocation_data

,site_desc,lat,lon
site_no,,,
1.0,SIGNALISED 4 WAY INTERSECTION,-34.921569,138.599436
2.0,SIGNALISED T-JUNCTION,-34.922999,138.599548
3.0,SIGNALISED 4 WAY INTERSECTION,-34.924385,138.599636
4.0,SIGNALISED 4 WAY INTERSECTION,-34.925770,138.599750
5.0,SIGNALISED 4 WAY INTERSECTION,-34.927124,138.599839
...,...,...,...
118.0,SIGNALISED T-JUNCTION,-34.921962,138.590743
119.0,SIGNALISED T-JUNCTION,-34.921231,138.585003
120.0,SIGNALISED PEDESTRIAN CROSSING,-34.936146,138.591253


In [18]:
json_data = []
SiteLocation_data = pd.read_csv('../data/0_Site_table.csv')

# Conversione in GeoJson- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
for _, row in SiteLocation_data.iterrows():
    data = OrderedDict()
    data['type'] = 'Feature'
    data['properties'] = {
        'site': row['site_no'],
        'site_desc' : row['site_desc']
    }
    data['geometry'] = {
        'type': 'Point',
        'coordinates': [float(row['lon']), float(row['lat'])]
    }
    json_data.append(data)
data = OrderedDict()
data['type'] = 'FeatureCollection'
data['features'] = json_data

with open('../data/0_Sensor_location.geojson', 'w') as f:
    f.write(json.dumps(data, sort_keys=False, indent=4))

# Plot mappa - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
marker_colors = {"SIGNALISED T-JUNCTION" : "red", "SIGNALISED 4 WAY INTERSECTION": "green", "SIGNALISED PEDESTRIAN CROSSING" :"blue"}

map = folium.Map(location=[-34.920369,138.6007179], zoom_start=14)
folium.GeoJson(
    data,
    name = "Site info",
    # zoom_on_click = True,
    popup=folium.GeoJsonPopup(fields=["site", "site_desc"]),
    style_function=lambda x: { #FIXME: markerColor doesn't work
        'markerColor': marker_colors.get(x['properties']['site_desc']),
    },
).add_to(map)
map.save(outfile='img/mappa.html')
display(map)

## Merge of traffic and weather data
Drop unused columns

In [7]:
tf_data.drop(columns={'lat','lon','t_detector_counts', 't_valid_counts', 't_error_counts', 'error_ratio'}, inplace=True)
tf_data.head()

,site_no,site_desc,date,hourly_traffic
0,1.0,SIGNALISED 4 WAY INTERSECTION,2010-01-01 00:00:00,4167.0
1,1.0,SIGNALISED 4 WAY INTERSECTION,2010-01-01 01:00:00,4000.0
2,1.0,SIGNALISED 4 WAY INTERSECTION,2010-01-01 02:00:00,3172.0
3,1.0,SIGNALISED 4 WAY INTERSECTION,2010-01-01 03:00:00,2558.0
4,1.0,SIGNALISED 4 WAY INTERSECTION,2010-01-01 04:00:00,2386.0


Aggiungi colonna giorno della settimana

In [8]:
Conv_DayToInt = {
    "Monday":   0,
    "Tuesday":  1,
    "Wednesday":2,
    "Thursday": 3,
    "Friday":   4,
    "Saturday": 5,
    "Sunday":   6
}

tf_data['day_of_week'] = pd.to_datetime(tf_data.date).dt.day_name()
tf_data['day_of_week'] = tf_data['day_of_week'].map(Conv_DayToInt)

del Conv_DayToInt

Addition of Holidays Column

In [9]:
AU_holidays = holidays.country_holidays('AU', years=range(2010, 2015))
AU_holidays.update({
    '2010-04-04': 'Easter Sunday',
    '2011-04-24': 'Easter Sunday',
    '2012-04-08': 'Easter Sunday',
    '2013-03-31': 'Easter Sunday',
    '2014-04-20': 'Easter Sunday',
    '2015-04-05': 'Easter Sunday',
})

AU_holidays = pd.DataFrame.from_dict(AU_holidays, orient='index').reset_index()
AU_holidays.columns = ['Date', 'Holiday']
AU_holidays['Date'] = pd.to_datetime(AU_holidays['Date'])

Bool_holidays = AU_holidays.copy()
Bool_holidays["Holiday"] = 1

tf_data['holiday'] = tf_data['date'].dt.date.map(Bool_holidays.set_index('Date')['Holiday']).fillna(0)
display(AU_holidays.groupby("Holiday").count())

del Bool_holidays, AU_holidays

,Date
Holiday,
ANZAC Day,5
ANZAC Day; Easter Monday,1
Australia Day,6
Boxing Day,6
Christmas Day,6
Easter Monday,5
Easter Sunday,6
Good Friday,6
New Year's Day,6


Merge...

In [12]:
print("Unione...")
Data_set = pd.merge(tf_data, wt_data, on="date")
Data_set.sort_values(by=['date', 'site_no'], inplace=True)

Data_set.reset_index(drop=True, inplace=True)
print("Fatto")

new_order = ['site_no', 'site_desc', 'date', 'hourly_traffic', 'temperature_2m', 'apparent_temperature', 'relative_humidity_2m', 'precipitation', 'wind_speed_10m', "cloud_cover", 'day_of_week', 'holiday']
Data_set = Data_set.reindex(columns=new_order)
display(Data_set)

del new_order, tf_data, wt_data

Unione...


Fatto


,site_no,site_desc,date,hourly_traffic,temperature_2m,apparent_temperature,relative_humidity_2m,precipitation,wind_speed_10m,cloud_cover,day_of_week,holiday
0,1.0,SIGNALISED 4 WAY INTERSECTION,2010-01-01 00:00:00,4167.0,23.539999,24.445896,52.424004,0.0,7.862518,0.9,4,1.0
1,2.0,SIGNALISED T-JUNCTION,2010-01-01 00:00:00,1932.0,23.539999,24.445896,52.424004,0.0,7.862518,0.9,4,1.0
2,3.0,SIGNALISED 4 WAY INTERSECTION,2010-01-01 00:00:00,2316.0,23.539999,24.445896,52.424004,0.0,7.862518,0.9,4,1.0
3,4.0,SIGNALISED 4 WAY INTERSECTION,2010-01-01 00:00:00,1699.0,23.539999,24.445896,52.424004,0.0,7.862518,0.9,4,1.0
4,5.0,SIGNALISED 4 WAY INTERSECTION,2010-01-01 00:00:00,1835.0,23.539999,24.445896,52.424004,0.0,7.862518,0.9,4,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5346523,118.0,SIGNALISED T-JUNCTION,2014-12-31 23:00:00,1845.0,25.689999,24.791649,36.087940,0.0,5.351785,22.2,2,0.0
5346524,119.0,SIGNALISED T-JUNCTION,2014-12-31 23:00:00,962.0,25.689999,24.791649,36.087940,0.0,5.351785,22.2,2,0.0
5346525,120.0,SIGNALISED PEDESTRIAN CROSSING,2014-12-31 23:00:00,210.0,25.689999,24.791649,36.087940,0.0,5.351785,22.2,2,0.0
5346526,121.0,SIGNALISED PEDESTRIAN CROSSING,2014-12-31 23:00:00,74.0,25.689999,24.791649,36.087940,0.0,5.351785,22.2,2,0.0


Save

In [13]:
Data_set.to_csv('../data/2_DateSet_NotNormalized.gz', index=False, compression='gzip')

## Data normalization post-merge
Data normalization is often a crucial phase in preparing data for machine learning. The main reasons are:

- *Improve algorithm convergence*: Many machine learning algorithms work better when data features have similar scales.
- *Reduce the influence of outliers*: Normalization makes the data more robust to outliers, which could otherwise negatively impact the model training.
- *Make models more interpretable*: In some cases, normalization can make the model coefficients more interpretable.

### Time of day and day of the week cycle encoding

In [14]:
day = 24*60*60
week = 7

timestamp_s = Data_set['date'].apply(lambda x: x.timestamp())

Data_set['hour_of_day(sin)'] = np.sin(timestamp_s * (2 * np.pi / day))
Data_set['hour_of_day(cos)'] = np.cos(timestamp_s * (2 * np.pi / day))

Data_set['day_of_week(sin)'] = np.sin(Data_set['day_of_week'] * (2 * np.pi / week))
Data_set['day_of_week(cos)'] = np.cos(Data_set['day_of_week'] * (2 * np.pi / week))

Data_set.drop(columns=["day_of_week"], inplace=True)

del timestamp_s, day, week
Data_set.head()

,site_no,site_desc,date,hourly_traffic,temperature_2m,apparent_temperature,relative_humidity_2m,precipitation,wind_speed_10m,cloud_cover,holiday,hour_of_day(sin),hour_of_day(cos),day_of_week(sin),day_of_week(cos)
0,1.0,SIGNALISED 4 WAY INTERSECTION,2010-01-01,4167.0,23.539999,24.445896,52.424004,0.0,7.862518,0.9,1.0,-4.416858e-12,1.0,-0.433884,-0.900969
1,2.0,SIGNALISED T-JUNCTION,2010-01-01,1932.0,23.539999,24.445896,52.424004,0.0,7.862518,0.9,1.0,-4.416858e-12,1.0,-0.433884,-0.900969
2,3.0,SIGNALISED 4 WAY INTERSECTION,2010-01-01,2316.0,23.539999,24.445896,52.424004,0.0,7.862518,0.9,1.0,-4.416858e-12,1.0,-0.433884,-0.900969
3,4.0,SIGNALISED 4 WAY INTERSECTION,2010-01-01,1699.0,23.539999,24.445896,52.424004,0.0,7.862518,0.9,1.0,-4.416858e-12,1.0,-0.433884,-0.900969
4,5.0,SIGNALISED 4 WAY INTERSECTION,2010-01-01,1835.0,23.539999,24.445896,52.424004,0.0,7.862518,0.9,1.0,-4.416858e-12,1.0,-0.433884,-0.900969


### Dummies of the type of intersection

In [15]:
Data_set = pd.get_dummies(Data_set, columns=['site_desc'], prefix='SD_')
Data_set.rename(columns={
    'SD__SIGNALISED 4 WAY INTERSECTION': 'SD_SIGNALISED_4_WAY_INTERSECTION',
    'SD__SIGNALISED PEDESTRIAN CROSSING': 'SD_SIGNALISED_PEDESTRIAN_CROSSING',
    'SD__SIGNALISED T-JUNCTION': 'SD_SIGNALISED_T_JUNCTION'
}, inplace=True)

Data_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5346528 entries, 0 to 5346527
Data columns (total 17 columns):
 #   Column                             Dtype         
---  ------                             -----         
 0   site_no                            float64       
 1   date                               datetime64[ns]
 2   hourly_traffic                     float64       
 3   temperature_2m                     float64       
 4   apparent_temperature               float64       
 5   relative_humidity_2m               float64       
 6   precipitation                      float64       
 7   wind_speed_10m                     float64       
 8   cloud_cover                        float64       
 9   holiday                            float64       
 10  hour_of_day(sin)                   float64       
 11  hour_of_day(cos)                   float64       
 12  day_of_week(sin)                   float64       
 13  day_of_week(cos)                   float64       
 14  SD

### Normalization of the remaining columns

In [19]:
# -- --- Normalizzazione traffico --- --
Data_set['hourly_traffic']       = Data_set['hourly_traffic'].apply(lambda x: x/1800)      # Veicoli/Ora

# -- --- Normalizzazione meteo --- --
Data_set['temperature_2m']       = Data_set['temperature_2m'].apply(lambda x: x/20)        # °C
Data_set['apparent_temperature'] = Data_set['apparent_temperature'].apply(lambda x: x/20)  # °C
Data_set['relative_humidity_2m'] = Data_set['relative_humidity_2m'].apply(lambda x: x/100) # %
Data_set['precipitation']        = Data_set['precipitation'].apply(lambda x: x/5)          # mm
Data_set['wind_speed_10m']       = Data_set['wind_speed_10m'].apply(lambda x: x/15)        # km/h
Data_set['cloud_cover']          = Data_set['cloud_cover'].apply(lambda x: x/100)          # %

# -- --- Normalizzazione info sulla data --- --
Data_set['hour_of_day(sin)']     = Data_set['hour_of_day(sin)'].apply(lambda x: (x+1)/2)   #
Data_set['hour_of_day(cos)']     = Data_set['hour_of_day(cos)'].apply(lambda x: (x+1)/2)   #

Data_set['day_of_week(sin)']     = Data_set['day_of_week(sin)'].apply(lambda x: (x+1)/2)   #
Data_set['day_of_week(cos)']     = Data_set['day_of_week(cos)'].apply(lambda x: (x+1)/2)   #

# -- --- info sui nuovi max, min --- --
Data_set.drop(columns=['site_no','date','hour_of_day(sin)','hour_of_day(cos)','day_of_week(sin)','day_of_week(sin)']).describe().T.drop(columns="count")

,mean,std,min,25%,50%,75%,max
hourly_traffic,0.686567,0.634843,0.000000,0.165000,0.514444,1.045000,6.123333
temperature_2m,0.833086,0.301612,0.117000,0.612000,0.774500,1.004500,2.019500
apparent_temperature,0.721823,0.344135,-0.049737,0.465404,0.656857,0.925835,2.071331
relative_humidity_2m,0.657317,0.197983,0.081371,0.516910,0.679468,0.820077,1.000000
precipitation,0.010049,0.049862,0.000000,0.000000,0.000000,0.000000,1.560000
wind_speed_10m,1.068027,0.514494,0.000000,0.686417,1.012845,1.399428,3.755674
cloud_cover,0.338667,0.311723,0.000000,0.030000,0.288000,0.558000,1.000000
holiday,0.021358,0.144575,0.000000,0.000000,0.000000,0.000000,1.000000
day_of_week(cos),0.500247,0.353493,0.049516,0.049516,0.388740,0.811745,1.000000


In [17]:
Data_set.to_csv('../data/3_DataSet_Normalized.gz',compression='gzip' , index=False)